In [2]:
import pandas as pd

In [6]:
df = pd.read_csv(r"C:\Users\alexv\PycharmProjects\FeatureSelection-MMO\Dados\train.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Columns: 195002 entries, beer_id to target
dtypes: float64(195001), int64(1)
memory usage: 299.0 MB


In [3]:
df.target.value_counts()

19.0    8
3.0     8
22.0    8
27.0    8
15.0    7
14.0    7
8.0     7
1.0     7
12.0    7
17.0    7
13.0    7
5.0     7
16.0    7
29.0    7
24.0    7
4.0     6
20.0    6
26.0    6
25.0    6
30.0    6
28.0    6
7.0     6
31.0    6
11.0    6
32.0    6
18.0    5
6.0     5
9.0     5
2.0     5
21.0    4
23.0    4
10.0    4
Name: target, dtype: int64

> Dataset um tanto desbalanceado

In [9]:
df.set_index("beer_id", drop=True, inplace=True)

In [10]:
X = df.drop(columns=['target'])
y = df.target

del df

In [11]:
# Vamos separar em treino e validação. Pegar 25% dos exemplos de cada cerveja
targets = y.unique()

In [12]:
X_val = X.loc[0:1].copy()
y_val = y.loc[0:1].copy()

# nesse trecho estou separando a base de treino em validacao e treino manualmente,
# pois deve-se haver 25% de individuos de cada classe na base de validacao.
for cerveja in targets:
    beers = X[y == cerveja]
    qtd_val = round(len(beers) * 0.25)
    val_sampled = beers.sample(qtd_val)
    y_val = pd.concat([y_val, y.loc[val_sampled.index]])
    X.drop(val_sampled.index, inplace=True)
    y.drop(val_sampled.index, inplace=True)
    X_val = pd.concat([X_val,val_sampled])

X_val.drop([0,1], inplace=True)
y_val.drop([0,1], inplace=True)

In [13]:
X.shape, X_val.shape

((144, 195000), (57, 195000))

In [16]:
y.shape, y_val.shape

((144,), (57,))

> Devemos fazer a divisão entre treino e validação antes de passar o over sampler, pois precisamos garantir representabilidade do dataset.

In [15]:
# balanceando com RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

In [17]:
balancer = RandomOverSampler(random_state=42)
X_train_rzd, y_train_rzd = balancer.fit_resample(X,y)

In [18]:
X_val_rzd, y_val_rzd = balancer.fit_resample(X_val, y_val)

In [19]:
y_val_rzd.value_counts()

25.0    2
28.0    2
26.0    2
10.0    2
9.0     2
20.0    2
4.0     2
17.0    2
12.0    2
1.0     2
22.0    2
21.0    2
3.0     2
5.0     2
13.0    2
14.0    2
2.0     2
7.0     2
8.0     2
18.0    2
31.0    2
11.0    2
19.0    2
15.0    2
6.0     2
23.0    2
27.0    2
16.0    2
29.0    2
24.0    2
32.0    2
30.0    2
Name: target, dtype: int64

In [20]:
y_train_rzd.value_counts()

25.0    6
28.0    6
30.0    6
14.0    6
24.0    6
26.0    6
10.0    6
9.0     6
20.0    6
15.0    6
4.0     6
17.0    6
12.0    6
1.0     6
22.0    6
3.0     6
5.0     6
32.0    6
13.0    6
2.0     6
7.0     6
8.0     6
19.0    6
18.0    6
31.0    6
11.0    6
6.0     6
23.0    6
27.0    6
16.0    6
29.0    6
21.0    6
Name: target, dtype: int64

In [21]:
# salvando em um csv
data_val = pd.concat([X_val_rzd, y_val_rzd], axis=1)
data = pd.concat([X_train_rzd, y_train_rzd], axis=1)

In [23]:
data.to_csv("../Dados/DataTrain.csv", index=False)
data_val.to_csv("../Dados/DataVal.csv", index=False)

In [26]:
data.target

0      25.0
1      28.0
2      29.0
3      16.0
4      27.0
       ... 
187    30.0
188    31.0
189    31.0
190    32.0
191    32.0
Name: target, Length: 192, dtype: float64

In [41]:
X_val_final = pd.read_csv('../Dados/DataVal.csv', dtype='float16')


In [42]:
X_val_final.to_parquet("DataVal.parquet", index=False, compression=None)

In [1]:
import pandas as pd

df = pd.read_parquet("../Dados/DataTrain.parquet", engine='fastparquet')

KeyboardInterrupt: 